In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def fetch_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    return table['Symbol'].tolist()

def calculate_ema(data, period):
    return data['Close'].ewm(span=period, adjust=False).mean()

def check_crossover(data):
    ema200 = calculate_ema(data, 200)
    last_3_bars = data.tail(3)
    crossover = (last_3_bars['Close'].iloc[0] <= ema200.iloc[-3]) and \
                (last_3_bars['Close'].iloc[-1] > ema200.iloc[-1])
    return crossover

def scan_stocks(tickers):
    end_time = datetime.now()
    start_time = end_time - timedelta(days=5)
    
    crossover_stocks = []
    
    for ticker in tickers:
        try:
            #stock = yf.Ticker(ticker)
            data =  yf.download(tickers=ticker, period='1y', interval='1d') #stock.history(start=start_time, end=end_time, interval="5m")
            
            if len(data) >= 200 and check_crossover(data):
                
                crossover_stocks.append(ticker)
        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")
    
    return crossover_stocks

if __name__ == "__main__":
    sp500_tickers = fetch_sp500_tickers()
    crossover_stocks = scan_stocks(sp500_tickers)
    
    print("Stocks with price crossing over EMA-200 in the last 3 bars (5-minute interval):")
    for stock in crossover_stocks:
        print(stock)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Stocks with price crossing over EMA-200 in the last 3 bars (5-minute interval):
BXP
CPT
DELL
DTE
DUK
ECL
EIX
EW
EQR
ESS
FRT
GPN
HIG
ICE
IVZ
IRM
NTAP
PRU
SO
TROW
TDG
VLTO


# SuperTrend Scanner

In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ta
from datetime import datetime, timedelta
a
def calculate_rma(df, length, column='Close'):
    """
    Calculate the Relative Moving Average (RMA) for a given DataFrame.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with OHLCV data
    length (int): The period for RMA calculation
    column (str): The column to use for calculation (default is 'Close')
    
    Returns:
    pandas.Series: RMA values
    """
    alpha = 1 / length
    
    # Initialize the RMA series with NaN values
    rma = pd.Series(index=df.index, dtype=float)
    
    # Calculate the first RMA value (same as SMA for the first 'length' periods)
    rma.iloc[length-1] = df[column].iloc[:length].mean()
    
    # Calculate subsequent RMA values
    for i in range(length, len(df)):
        rma.iloc[i] = (df[column].iloc[i] * alpha) + (rma.iloc[i-1] * (1 - alpha))
    return rma

def calculate_atr(data, period=10):
    """
    Calculate the Average True Range (ATR) using TradingView-like logic.
    """
    # True Range (TR) calculations
    data['TR1'] = data['High'] - data['Low']
    data['TR2'] = abs(data['High'] - data['Close'].shift(1))
    data['TR3'] = abs(data['Low'] - data['Close'].shift(1))
    data['TR'] = data[['TR1', 'TR2', 'TR3']].max(axis=1)

    # Average True Range (ATR) as Relative Moving Average (RMA) of TR
    #data['ATR'] =  data['TR'].rolling(window=period).mean()  #calculate_rma(data, 'TR', period)
    # Calculate RMA

    data['ATR'] = calculate_rma(data, length = 10, column = 'TR')
    #data['ATR'] =  calculate_rma(data, 'TR', period)
    return data

# 2. SuperTrend Calculation
def calculate_supertrend(df, period=10, multiplier=3.0, change_atr=True):

    print(df.columns)
    # Calculate ATR
    df = calculate_atr(df, period=period)
    print("ATR***")
    high = df['High']
    low = df['Low']
    close = df['Close']
    atr = df['ATR']
    print("step3")

    # Calculate True Range
    #tr1 = pd.DataFrame(high - low)
    #tr2 = pd.DataFrame(abs(high - close.shift(1)))
    #tr3 = pd.DataFrame(abs(low - close.shift(1)))
    #frames = [tr1, tr2, tr3]
    #tr = pd.concat(frames, axis=1, join='inner').max(axis=1)
    
    # Calculate ATR
    #if change_atr:
    #    atr = tr.ewm(alpha=1/period, min_periods=period).mean()
    #else:
    #    atr = tr.rolling(window=period).mean()
    
    # Calculate SuperTrend
    hl2 = (high + low) / 2
    
    # Basic bands
    final_upperband = hl2 - (multiplier * atr)
    final_lowerband = hl2 + (multiplier * atr)

    print("step4")
    # Initialize Supertrend columns
    supertrend = pd.DataFrame(index=df.index)
    supertrend['up'] = final_upperband
    supertrend['down'] = final_lowerband
    supertrend['trend'] = 1
    
    # Calculate SuperTrend
    for i in range(1, len(df.index)):
        curr = df.index[i]
        prev = df.index[i-1]
        
        # Up trend
        if close[curr] > supertrend.loc[prev, 'up']:
            supertrend.loc[curr, 'up'] = max(supertrend.loc[prev, 'up'], final_upperband[curr])
        else:
            supertrend.loc[curr, 'up'] = final_upperband[curr]
            
        # Down trend
        if close[curr] < supertrend.loc[prev, 'down']:
            supertrend.loc[curr, 'down'] = min(supertrend.loc[prev, 'down'], final_lowerband[curr])
        else:
            supertrend.loc[curr, 'down'] = final_lowerband[curr]
            
        # Trend
        if close[curr] > supertrend.loc[prev, 'down']:
            supertrend.loc[curr, 'trend'] = 1
        elif close[curr] < supertrend.loc[prev, 'up']:
            supertrend.loc[curr, 'trend'] = -1
        else:
            supertrend.loc[curr, 'trend'] = supertrend.loc[prev, 'trend']
            
    # Generate signals
    supertrend['buy_signal'] = (supertrend['trend'] == 1) & (supertrend['trend'].shift(1) == -1)
    supertrend['sell_signal'] = (supertrend['trend'] == -1) & (supertrend['trend'].shift(1) == 1)
    
    return supertrend

def calculateQQE(rsiLength, smoothingFactor, qqeFactor, source):
    wildersLength = rsiLength * 2 - 1

    delta = source.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    up_avg = up.ewm(alpha=1/rsiLength, adjust=False).mean()
    down_avg = down.ewm(alpha=1/rsiLength, adjust=False).mean()
    rs = up_avg / down_avg
    rsi = 100 - (100 / (1 + rs))
    
    smoothedRsi = rsi.ewm(span=smoothingFactor, adjust=False).mean()

    atrRsi = (smoothedRsi - smoothedRsi.shift(1)).abs()
    smoothedAtrRsi = atrRsi.ewm(span=wildersLength, adjust=False).mean()
    dynamicAtrRsi = smoothedAtrRsi * qqeFactor

    longBand = pd.Series(index=source.index)
    shortBand = pd.Series(index=source.index)
    trendDirection = pd.Series(index=source.index, dtype='int')

    longBand.iloc[0] = smoothedRsi.iloc[0]
    shortBand.iloc[0] = smoothedRsi.iloc[0]
    trendDirection.iloc[0] = 0

    for i in range(1, len(source)):
        atrDelta = dynamicAtrRsi.iloc[i]
        newLongBand = smoothedRsi.iloc[i] - atrDelta
        newShortBand = smoothedRsi.iloc[i] + atrDelta

        prevLongBand = longBand.iloc[i - 1]
        prevShortBand = shortBand.iloc[i - 1]
        prevSmoothedRsi = smoothedRsi.iloc[i - 1]

        if prevSmoothedRsi > prevLongBand and smoothedRsi.iloc[i] > prevLongBand:
            longBand.iloc[i] = max(prevLongBand, newLongBand)
        else:
            longBand.iloc[i] = newLongBand

        if prevSmoothedRsi < prevShortBand and smoothedRsi.iloc[i] < prevShortBand:
            shortBand.iloc[i] = min(prevShortBand, newShortBand)
        else:
            shortBand.iloc[i] = newShortBand

        if smoothedRsi.iloc[i] > prevShortBand and prevSmoothedRsi <= prevShortBand:
            trendDirection.iloc[i] = 1
        elif smoothedRsi.iloc[i] < prevLongBand and prevSmoothedRsi >= prevLongBand:
            trendDirection.iloc[i] = -1
        else:
            trendDirection.iloc[i] = trendDirection.iloc[i - 1]

    qqeTrendLine = pd.Series(index=source.index)
    for i in range(len(source)):
        if trendDirection.iloc[i] == 1:
            qqeTrendLine.iloc[i] = longBand.iloc[i]
        else:
            qqeTrendLine.iloc[i] = shortBand.iloc[i]

    return qqeTrendLine, smoothedRsi

def qqe_mod(data):

    # Define QQE Indicator Inputs
    rsiLengthPrimary = 6
    rsiSmoothingPrimary = 5
    qqeFactorPrimary = 3.0
    thresholdPrimary = 3.0
    sourcePrimary = data['Close']

    rsiLengthSecondary = 6
    rsiSmoothingSecondary = 5
    qqeFactorSecondary = 1.61
    thresholdSecondary = 3.0
    sourceSecondary = data['Close']

    bollingerLength = 50
    bollingerMultiplier = 0.35

    # Main Calculations
    primaryQQETrendLine, primaryRSI = calculateQQE(rsiLengthPrimary, rsiSmoothingPrimary, qqeFactorPrimary, sourcePrimary)
    secondaryQQETrendLine, secondaryRSI = calculateQQE(rsiLengthSecondary, rsiSmoothingSecondary, qqeFactorSecondary, sourceSecondary)
    
    primaryQQETrendLineAdj = primaryQQETrendLine - 50
    bollingerBasis = primaryQQETrendLineAdj.rolling(window=bollingerLength).mean()
    bollingerDeviation = bollingerMultiplier * primaryQQETrendLineAdj.rolling(window=bollingerLength).std()
    bollingerUpper = bollingerBasis + bollingerDeviation
    bollingerLower = bollingerBasis - bollingerDeviation
    
    primaryRSIAdj = primaryRSI - 50
    secondaryRSIAdj = secondaryRSI - 50

    # Add new columns to the DataFrame
    data['Primary RSI'] = primaryRSIAdj
    data['Secondary RSI'] = secondaryRSIAdj
    data['Bollinger Upper'] = bollingerUpper
    data['Bollinger Lower'] = bollingerLower

    # Add signal column
    data['qqe_Signal'] = np.where((secondaryRSIAdj > thresholdSecondary) & (primaryRSIAdj > bollingerUpper), 'Up',
                              np.where((secondaryRSIAdj < -thresholdSecondary) & (primaryRSIAdj < bollingerLower), 'Down', None))

    #data.drop(['Bollinger Upper','Bollinger Lower','Primary RSI','Secondary RSI'])
    data = data.drop(columns=['Bollinger Upper','Bollinger Lower','Primary RSI','Secondary RSI']) 
    return data

def fetch_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    return table['Symbol'].tolist()

def calculate_ema(data, period):
    return data['Close'].ewm(span=period, adjust=False).mean()

def check_crossover(data):
    ema200 = calculate_ema(data, 200)
    last_3_bars = data.tail(3)
    crossover = (last_3_bars['Close'].iloc[0] <= ema200.iloc[-3]) and \
                (last_3_bars['Close'].iloc[-1] > ema200.iloc[-1])
    return crossover

def scan_stocks(tickers):
    end_time = datetime.now()
    start_time = end_time - timedelta(days=5)
    
    crossover_stocks = []
    
    for ticker in tickers:
        try:
            #stock = yf.Ticker(ticker)
            data =  yf.download(tickers=ticker, period='1y', interval='1d') #stock.history(start=start_time, end=end_time, interval="5m")
            df = data
            st = calculate_supertrend(
                data,
                period=10,          # ATR Period
                multiplier=3.0,     # ATR Multiplier
                change_atr=True     # Use EMA for ATR calculation
            )
        
            # Merge with original data
            df['SuperTrend_Up'] = np.where(st['trend'] == 1, st['up'], np.nan)
            df['SuperTrend_Down'] = np.where(st['trend'] == -1, st['down'], np.nan)
            df['Trend'] = st['trend']
            df['Buy_Signal'] = st['buy_signal']
            df['Sell_Signal'] = st['sell_signal']
        
            print(df.columns)
            df['ema50'] = df['Close'].ewm(span=50, adjust=False).mean()
            df['ema100'] = df['Close'].ewm(span=100, adjust=False).mean()
            df['ema200'] = df['Close'].ewm(span=200, adjust=False).mean()
            #df[['maDf','maAcc','inChannel']] = ema_slope(df)
        
            df = qqe_mod(df)
            # Calculate the absolute and percentage gap between EMA50 and EMA200            

            row = df.iloc[-1]    
           
            #if len(df) >= 200 and  ((row['Trend'] == 1) and (row['qqe_Signal'] == 'Up')) :  #( row['Buy_Signal'] == True)
            if len(df) >= 200 and  ((row['Trend'] == 1) and (row['Buy_Signal'] == True)) :  #( row['Buy_Signal'] == True)
                crossover_stocks.append(ticker)
        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")
    
    return crossover_stocks

if __name__ == "__main__":
    sp500_tickers = fetch_sp500_tickers()
    crossover_stocks = scan_stocks(sp500_tickers)
    
    print("Stocks with price crossing over EMA-200 in the last 3 bars (5-minute interval):")
    for stock in crossover_stocks:
        print(stock)

[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
ATR***
step3
step4
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TR1', 'TR2',
       'TR3', 'TR', 'ATR', 'SuperTrend_Up', 'SuperTrend_Down', 'Trend',
       'Buy_Signal', 'Sell_Signal'],
      dtype='object')


[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
ATR***
step3
step4
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TR1', 'TR2',
       'TR3', 'TR', 'ATR', 'SuperTrend_Up', 'SuperTrend_Down', 'Trend',
       'Buy_Signal', 'Sell_Signal'],
      dtype='object')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TR1', 'TR2',
       'TR3', 'TR', 'ATR', 'SuperTrend_Up', 'SuperTrend_Down', 'Trend',
       'Buy_Signal', 'Sell_Signal'],
      dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
ATR***
step3
step4
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TR1', 'TR2',
       'TR3', 'TR', 'ATR', 'SuperTrend_Up', 'SuperTrend_Down', 'Trend',
       'Buy_Signal', 'Sell_Signal'],
      dtype='object')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
ATR***
step3
step4
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TR1', 'TR2',
       'TR3', 'TR', 'ATR', 'SuperTrend_Up', 'SuperTrend_Down', 'Trend',
       'Buy_Signal', 'Sell_Signal'],
      dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
ATR***
step3
step4


[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TR1', 'TR2',
       'TR3', 'TR', 'ATR', 'SuperTrend_Up', 'SuperTrend_Down', 'Trend',
       'Buy_Signal', 'Sell_Signal'],
      dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
ATR***
step3
step4
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TR1', 'TR2',
       'TR3', 'TR', 'ATR', 'SuperTrend_Up', 'SuperTrend_Down', 'Trend',
       'Buy_Signal', 'Sell_Signal'],
      dtype='object')
Stocks with price crossing over EMA-200 in the last 3 bars (5-minute interval):


In [29]:
crossover_stocks

[]

In [82]:
#!pip install Yahoo-ticker-downloader
!YahooTickerDownloader.py -E NYSE

Checking if we can resume a old download session
No old downloader found on disk
Starting a new session
Execution of script halted due to robots.txt


In [88]:
!pip install pytickersymbols

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 3.9 MB/s eta 0:00:00


In [90]:
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
countries = stock_data.get_all_countries()
indices = stock_data.get_all_indices()
industries = stock_data.get_all_industries()


In [94]:
indices

['S&P 600',
 'OMX Stockholm 30',
 'MDAX',
 'CDAX',
 'NASDAQ 100',
 'S&P 500',
 'Switzerland 20',
 'DOW JONES',
 'S&P 100',
 'DAX',
 'FTSE 100',
 'BEL 20',
 'MOEX',
 'EURO STOXX 50',
 'TECDAX',
 'AEX',
 'IBEX 35',
 'CAC Mid 60',
 'OMX Helsinki 25',
 'SDAX',
 'CAC 40']

In [100]:
import requests
import pandas as pd

def fetch_mega_cap_tickers():
    url = "https://stockanalysis.com/list/mega-cap-stocks/"
    
    try:
        # Fetch the HTML content
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse the HTML table
        tables = pd.read_html(response.text)
        
        # The table we want is typically the first one
        mega_cap_df = tables[0]
        
        # Extract the 'Symbol' column
        tickers = mega_cap_df['Symbol'].tolist()
        
        return tickers
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []

# Fetch and print the ticker symbols
mega_cap_tickers = fetch_mega_cap_tickers()
print("Mega-cap stock tickers:")
print(mega_cap_tickers)

Mega-cap stock tickers:
['AAPL', 'NVDA', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'AVGO', 'BRK.B', 'TSM', 'WMT', 'LLY', 'JPM', 'V', 'MA', 'ORCL', 'XOM', 'UNH', 'COST', 'PG', 'NFLX', 'HD', 'NVO', 'JNJ', 'BAC', 'CRM', 'ABBV', 'SAP', 'ASML', 'KO', 'TMUS', 'CVX', 'MRK', 'CSCO', 'TM', 'WFC', 'NOW', 'ACN', 'BX', 'AXP', 'PEP', 'MCD', 'IBM', 'AMD', 'DIS', 'MS', 'LIN', 'TMO']


/var/folders/9b/ckc59bc918g30hv5rlmd9yv00000gn/T/ipykernel_26530/1588485997.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
